In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle 

Path.cwd()

PosixPath('/Users/huynhphuong/Desktop/CAMCOS-after-semester')

## Analyze data from September 6 and 7

In [ ]:
sep6 = '/Users/huynhphuong/Desktop/CAMCOS-after-semester/Data/Sep-06.csv'
sep6 = pd.read_csv(sep6, sep='\t', dtype ={"curblocknumber":int,'gas':float, 
                                           'gasprice':float,'type':float,'maxfeepergas':float,
                                           'basefeepergas':float,'gasused':float},low_memory=False)

sep7 = '/Users/huynhphuong/Desktop/CAMCOS-after-semester/Data/Sep-07.csv'
sep7 = pd.read_csv(sep7, sep='\t', dtype ={"curblocknumber":int,'gas':float, 
                                           'gasprice':float,'type':float,'maxfeepergas':float,
                                           'basefeepergas':float,'gasused':float},low_memory=False)
df = pd.concat([sep6, sep7],ignore_index = True)
df

In [ ]:
table = df.copy()
df.columns

# 1. Column information 
also found at https://docs.blocknative.com/mempool-data-program#data-schema

#### 1. Transaction status: 
- rejected - denied entry into the mempool 
- evicted -  exitting the mempool 
- confirmed - finalized on chain 
- pending - pending in the mempool
- speedup - replacing existing mempool tx
- cancel - replacing existing mempool tx
- failed - finalized on chain 

#### 2. For each status, there are corresponding columns
- rejected --> rejectionreason 
- evicted --> dropreason 
- confirmed --> gasused 
- pending --> gasused = -1 
- speedup --> replace 
- cancel --> replace 
- failed --> failurereason 

#### 3. Dypes
    - detecttime ( datetime) - timestamp that txn was detected in the mempool
    - hash (string) -  unique indentifier hash for a txn
    - status (string) - status of a txn
    - region (string) - region of the node that detected the txn (us-east-1, eu-central-1, ap-southeast-1)
    - reorg (string) - refer to block hash of the reorg, if there was one
    - replace (string) - if a txn was replaced (speedup/cancel), show the txn hash of the replacement
    - curblocknumber (int)- the block number the event was detected in 
    - failurereason - reason of a failed txn
    - blockspending (int) - number of blocks that txn a txn was waiting to get on chain after finalized (confirmed/failed)
    - time spending (big int) - time in milisecinds that txn was waiting to get on chain
    - nonce (int) - unique number count the number of txns sent from a given address
    - gas (int) - a txn fee to incentivize miners (denom is gwei or 1e9ETH)(also the gas limit for a txn set by the users)
    - gasprice (int) - price offered to miner to purchase gas (denom is gwei or 1e9ETH) (only for txns type 0 and 1)
    - value(int) - amount of ETH (denom is gwei) transferred
    - toadress (string)
    - fromadress (string)
    - input (string) - additional data that can be attached to a transaction. This field can be used to tell a smart contract to execute a function
    - network (string) - ETH network used
    - type (int) - type 0,1,2
    - maxpriorityfeepergas (int) - max possible tip (actual tip = maxfee - basefee) (denom is gwei)
    - maxfeepergas (int) - max possible gas a sender is willing to pay including basefee and tip (why must be lower than the maxpriorityfeepergas?)
    - basefeepergas (int) - burned fee (denom is gwei)
    - dropreason (string)
    - rejectionreason (string)
    - stuck - no info, dropped
    - gasused (int) - actuan gas consumed (denom is gwei)
    - detect_date (string) 

#### 4. Current dtypes:
    detecttime               object 
    hash                     object
    status                   object
    region                   object
    reorg                    string
    replace                  object
    curblocknumber            int64
    failurereason            object
    blockspending            object
    timepending              object
    nonce                    object
    gas                     float64
    gasprice                float64
    value                    object
    toaddress                object
    fromaddress              object
    input                    object
    network                  object
    type                    float64
    maxpriorityfeepergas     object
    maxfeepergas            float64
    basefeepergas           float64
    dropreason               object
    rejectionreason          object
    stuck                    object
    gasused                 float64
    detect_date              object
    
#### 5. Dtypes needed to be fixed (maybe data entry error)
    - region: drop rows that are not the three region 
    - blockspending
    - timepending
    - nonce
    - maxpriorityfeepergas
    
#### 6. Drop columns
    - stuck: no information
    - network: only one value 'main, 
    - gasprice: if analyzing post EIP txns (type 2 txns)

#### 7. Some Observations
    - There are some duplicate hash for the same txns

# 2. Processing Data
### 2.1 drop columns and data entry error rows

In [ ]:
table = table.drop(columns = ["stuck","network",'gasprice'])
table = table[table.region != '7']
# table = table.reset_index(drop=True)
table.head(5)

### 2.2 Fix dtypes

In [ ]:
table.maxpriorityfeepergas = table.maxpriorityfeepergas.astype(float)
table.blockspending = table.blockspending.astype(float)
table.timepending = table.timepending.astype(float)
table.nonce = table.nonce.astype(float)
table.dtypes

# 3. Filtering data
### 3.1 
- Only US-east-1 region
- Only confirmed/failed txns
- Only type 2 txns
- Remove duplicate (some confirmed txns have dumplicated rows)

In [ ]:
table = table[(table["region"] == "us-east-1") & ((table["status"] == "confirmed" |table["status"] == "failed")) & (table["type"] == 2)]
print("Number of duplicate: ", sum(table.hash.duplicated())) # check for reason of duplication
table = table.drop_duplicates(subset=['hash'], keep='first')
table = table.reset_index(drop = True)
table.head(10)

### 3.2 Exporting the csv file

In [ ]:
# table.to_csv("confirmed-txn.csv")

### Calculating mean gas, basefee, and mean max priority fee for each block

In [ ]:
# # getting the mean gas of each txns in each block
# f = table.groupby(["curblocknumber"]).gasused.mean()
# f = f.reset_index()

# # getting the basefeepergas for each block
# g = table.groupby(["curblocknumber"]).basefeepergas.mean()
# g = g.reset_index()

# # getting the mean of max prioprity fee for each block
# z = table.groupby(["curblocknumber"]).maxpriorityfeepergas.mean()
# z = z.reset_index()

# # getting the block size
# r = table.groupby(["curblocknumber"]).gasused.sum()
# r = r.reset_index()

# z["mean_gasused"] = f["gasused"]
# z["basefeepergas"] = g["basefeepergas"]
# z["blockbasefee"] = g["basefeepergas"]*(10**(-9))  #converting to gwei
# z["blocksize"] = r["gasused"]
# z["block_reward"] = z["maxpriorityfeepergas"]*z["blocksize"]*(10**(-18))
# z.head(10)

In [ ]:
# KEEP this cell, for counting duplicated hash values

#MyList = s[s["curblocknumber"]==18080834]
# MyList = MyList["hash"].tolist()

# my_dict = {i:MyList.count(i) for i in MyList}
# my_dict

In [ ]:
## KEEP this cell, for blocks that failed (null value)
# for i in list(r["curblocknumber"]):
#     if i not in list(z["curblocknumber"].unique()):
#         print(i)
# r

In [ ]:
# fig, ax = plt.subplots(4, figsize=(10,10))

# # using padding to create gap between plots
# fig.tight_layout(pad=4.0)   
# # sns.set_style('darkgrid')

# ax[0].plot(z["block_reward"])
# ax[0].set_title("block-reward (ETH)")
# # ax[0].plot(z['block_reward'].rolling(window=20, center=True).mean())
# ax[1].plot(z["mean_gasused"],color = "orange");
# ax[1].set_title("mean txns gas for each block");
# # ax[1].plot(z['mean_gasused'].rolling(window=100, center=True).mean())
# ax[2].plot(z["blockbasefee"],color = "green");
# ax[2].set_title("basefeepergas");
# ax[2].plot(z['blockbasefee'].rolling(window=100, center=True).mean(),color = 'red');
# ax[3].plot(z["blocksize"]);
# # ax[3].stackplot(range(0,len(z),1),z["blocksize"], color = "red");
# ax[3].set_title("block size");
# # ax[3].plot(z['blocksize'].rolling(window=10, center=True).mean());


There are two block that has gas over 30M

In [ ]:
# # getting the base fee of each block
# each_hour = table.groupby(["hour"]).basefeepergas.sum()
# each_hour = each_hour.reset_index()
# each_hour["blockbasefee"] = each_hour["basefeepergas"]*(10**(-9))

# # table["maxpriorityfeepergas"] = table["maxpriorityfeepergas"].astype(int)

# # getting the mean of max prioprity fee for each block
# f = table.groupby(["hour"]).maxpriorityfeepergas.mean()
# f = f.reset_index()

# each_hour["maxpriorityfeepergas"] = f["maxpriorityfeepergas"]

# each_hour.head(3)

In [ ]:
# fig, ax = plt.subplots(2, figsize=(10, 4))

# # using padding to create gap between plots
# fig.tight_layout(pad=4.0)   
 
# # sns.set_style('darkgrid')
# # ax[0].plot(each_hour["reward"])
# # ax[0].set_title("block reward (ETH)")
# ax[1].plot(each_hour["blockbasefee"])
# ax[1].set_title("base fee (gwei)")
# # ax[1].plot(each_hour['blockbasefee'].rolling(window=100, center=True).mean())